In [1]:
import pandas as pd
from difflib import get_close_matches

In [2]:
df1 = pd.read_csv('../data/cities_loc.csv')
df2 = pd.read_csv('../data/cleaned_communes_francaises.csv')

In [3]:
def clean_city_name(city_name):
    allowed_chars = set(" -'")  # espaces, tirets, apostrophes
    cleaned = []

    for ch in city_name:
        if ch.isalpha() or ch in allowed_chars:
            cleaned.append(ch)

    cleaned_name = "".join(cleaned)
    cleaned_name = " ".join(cleaned_name.split())  # retirer espaces multiples
    return cleaned_name

df1['cleaned_city_name'] = df1['city'].apply(clean_city_name)
df1.head()


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper,cleaned_city_name
0,Paris,48.8567,2.3522,France,FR,Île-de-France,primary,11060000,2148271,Paris
1,Bordeaux,44.8400,-0.5800,France,FR,Nouvelle-Aquitaine,admin,994920,994920,Bordeaux
2,Marseille,43.2964,5.3700,France,FR,Provence-Alpes-Côte d’Azur,admin,873076,873076,Marseille
3,Lyon,45.7600,4.8400,France,FR,Auvergne-Rhône-Alpes,admin,522250,522250,Lyon
4,Toulouse,43.6045,1.4440,France,FR,Occitanie,admin,504078,504078,Toulouse


In [4]:
df1.drop_duplicates(subset=['cleaned_city_name'], inplace=True)

In [5]:
# write a function that helps me find close matches and replace them in df1
def replace_with_closest_match(name, reference_list, cutoff=0.8):
    close_matches = get_close_matches(name, reference_list, n=1, cutoff=cutoff)
    if close_matches:
        return close_matches[0]
    return name  # return the original name if no close match is found  
df1['cleaned_city_name'] = df1['cleaned_city_name'].apply(lambda x: replace_with_closest_match(x, df2['cleaned_city_name']))

In [6]:
df_merged = pd.merge(df1, df2, left_on='cleaned_city_name', right_on='cleaned_city_name', how='left')

In [7]:
df_merged.isna().sum()

city                      0
lat                       0
lng                       0
country                   0
iso2                      0
admin_name                0
capital                 506
population                0
population_proper         0
cleaned_city_name         0
Commune                   7
Département (numéro)      7
Département (nom)         7
Région                    7
dtype: int64

In [8]:
df_merged.drop(columns=['capital'], inplace=True)

In [9]:
df_merged.head()

,city,lat,lng,country,iso2,admin_name,population,population_proper,cleaned_city_name,Commune,Département (numéro),Département (nom),Région
0,Paris,48.8567,2.3522,France,FR,Île-de-France,11060000,2148271,Paris,Paris 01,75,Paris,Île-de-France
1,Bordeaux,44.8400,-0.5800,France,FR,Nouvelle-Aquitaine,994920,994920,Bordeaux,Bordeaux,33,Gironde,Nouvelle-Aquitaine
2,Marseille,43.2964,5.3700,France,FR,Provence-Alpes-Côte d’Azur,873076,873076,Marseille,Marseille 01,13,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur
3,Lyon,45.7600,4.8400,France,FR,Auvergne-Rhône-Alpes,522250,522250,Lyon,Lyon 01,69,Rhône,Auvergne-Rhône-Alpes
4,Toulouse,43.6045,1.4440,France,FR,Occitanie,504078,504078,Toulouse,Toulouse,31,Haute-Garonne,Occitanie


In [10]:
df_merged[df_merged['Commune'].isna()] 

,city,lat,lng,country,iso2,admin_name,population,population_proper,cleaned_city_name,Commune,Département (numéro),Département (nom),Région
262,Cherbourg,49.6300,-1.6200,France,FR,Normandie,25370,25370,Cherbourg,NaN,NaN,NaN,NaN
342,Hellemmes-Lille,50.6167,3.1167,France,FR,Hauts-de-France,18437,18437,Hellemmes-Lille,NaN,NaN,NaN,NaN
348,Rosendaël,51.0413,2.4037,France,FR,Hauts-de-France,18272,18272,Rosendaël,NaN,NaN,NaN,NaN
377,Petite-Synthe,51.0194,2.3467,France,FR,Hauts-de-France,15911,15911,Petite-Synthe,NaN,NaN,NaN,NaN
466,Flers-lez-Lille,50.6333,3.1333,France,FR,Hauts-de-France,10561,10561,Flers-lez-Lille,NaN,NaN,NaN,NaN
525,Terrenoire,45.4343,4.4377,France,FR,Auvergne-Rhône-Alpes,8317,8317,Terrenoire,NaN,NaN,NaN,NaN
615,Fort-Mardyck,51.0310,2.3061,France,FR,Hauts-de-France,3426,3426,Fort-Mardyck,NaN,NaN,NaN,NaN


In [11]:
closest_match = replace_with_closest_match(df_merged[df_merged['Commune'].isna()],df2['cleaned_city_name'])

In [12]:
closest_match

,city,lat,lng,country,iso2,admin_name,population,population_proper,cleaned_city_name,Commune,Département (numéro),Département (nom),Région
262,Cherbourg,49.6300,-1.6200,France,FR,Normandie,25370,25370,Cherbourg,NaN,NaN,NaN,NaN
342,Hellemmes-Lille,50.6167,3.1167,France,FR,Hauts-de-France,18437,18437,Hellemmes-Lille,NaN,NaN,NaN,NaN
348,Rosendaël,51.0413,2.4037,France,FR,Hauts-de-France,18272,18272,Rosendaël,NaN,NaN,NaN,NaN
377,Petite-Synthe,51.0194,2.3467,France,FR,Hauts-de-France,15911,15911,Petite-Synthe,NaN,NaN,NaN,NaN
466,Flers-lez-Lille,50.6333,3.1333,France,FR,Hauts-de-France,10561,10561,Flers-lez-Lille,NaN,NaN,NaN,NaN
525,Terrenoire,45.4343,4.4377,France,FR,Auvergne-Rhône-Alpes,8317,8317,Terrenoire,NaN,NaN,NaN,NaN
615,Fort-Mardyck,51.0310,2.3061,France,FR,Hauts-de-France,3426,3426,Fort-Mardyck,NaN,NaN,NaN,NaN


In [13]:
df2[df2["Commune"].str.contains("Cherbourg", na=False)]

,Commune,Département (numéro),Département (nom),Région,cleaned_city_name
7249,Cherbourg-en-Cotentin,50,Manche,Normandie,Cherbourg-en-Cotentin


In [14]:
# Vérifier les similarités pour les villes non matchées
from difflib import SequenceMatcher

missing_cities = df_merged[df_merged['Commune'].isna()]['cleaned_city_name'].tolist()

for city in missing_cities:
    matches = get_close_matches(city, df2['cleaned_city_name'], n=3, cutoff=0.6)
    print(f"\n{city}:")
    for match in matches:
        similarity = SequenceMatcher(None, city, match).ratio()
        print(f"  - {match} (similarité: {similarity:.2f})")


Cherbourg:
  - Cleebourg (similarité: 0.78)
  - Châteaubourg (similarité: 0.76)
  - Sebourg (similarité: 0.75)

Hellemmes-Lille:
  - Helleville (similarité: 0.72)
  - Melleville (similarité: 0.64)
  - Hermeville (similarité: 0.64)

Rosendaël:
  - Rosenau (similarité: 0.75)
  - Rosel (similarité: 0.71)
  - Ronsenac (similarité: 0.71)

Petite-Synthe:
  - Petite-Île (similarité: 0.70)
  - Petit-Fayt (similarité: 0.70)
  - Petitefontaine (similarité: 0.67)

Flers-lez-Lille:
  - Villers-le-Tilleul (similarité: 0.73)
  - Villers-la-Ville (similarité: 0.71)
  - Marquette-lez-Lille (similarité: 0.65)

Terrenoire:
  - Terrefondrée (similarité: 0.73)
  - Vendoire (similarité: 0.67)
  - Terroles (similarité: 0.56)

Fort-Mardyck:
  - Le Port-Marly (similarité: 0.64)
  - Lorry-Mardigny (similarité: 0.62)


In [15]:
# Mapping manuel des anciennes communes vers les nouvelles
manual_mapping = {
    'Cherbourg': 'Cherbourg-en-Cotentin',
    'Hellemmes-Lille': 'Lille',
    'Rosendaël': 'Dunkerque',
    'Petite-Synthe': 'Dunkerque',
    'Flers-lez-Lille': 'Villeneuve-d\'Ascq',
    'Terrenoire': 'Saint-Étienne',
    'Fort-Mardyck': 'Dunkerque'
}

# Appliquer le mapping manuel aux villes non matchées
for old_name, new_name in manual_mapping.items():
    df_merged.loc[df_merged['cleaned_city_name'] == old_name, 'cleaned_city_name'] = new_name

# Refaire le merge pour ces villes
missing_indices = df_merged[df_merged['Commune'].isna()].index
for idx in missing_indices:
    city_name = df_merged.loc[idx, 'cleaned_city_name']
    match_data = df2[df2['cleaned_city_name'] == city_name]
    if not match_data.empty:
        df_merged.loc[idx, 'Commune'] = match_data.iloc[0]['Commune']
        df_merged.loc[idx, 'Département (numéro)'] = match_data.iloc[0]['Département (numéro)']
        df_merged.loc[idx, 'Département (nom)'] = match_data.iloc[0]['Département (nom)']
        df_merged.loc[idx, 'Région'] = match_data.iloc[0]['Région']

# Vérifier
df_merged.isna().sum()

city                    0
lat                     0
lng                     0
country                 0
iso2                    0
admin_name              0
population              0
population_proper       0
cleaned_city_name       0
Commune                 0
Département (numéro)    0
Département (nom)       0
Région                  0
dtype: int64

In [20]:
df_merged.head()

,city,lat,lng,country,iso2,admin_name,population,population_proper,cleaned_city_name,Commune,Département (numéro),Département (nom),Région
0,Paris,48.8567,2.3522,France,FR,Île-de-France,11060000,2148271,Paris,Paris 01,75,Paris,Île-de-France
1,Bordeaux,44.8400,-0.5800,France,FR,Nouvelle-Aquitaine,994920,994920,Bordeaux,Bordeaux,33,Gironde,Nouvelle-Aquitaine
2,Marseille,43.2964,5.3700,France,FR,Provence-Alpes-Côte d’Azur,873076,873076,Marseille,Marseille 01,13,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur
3,Lyon,45.7600,4.8400,France,FR,Auvergne-Rhône-Alpes,522250,522250,Lyon,Lyon 01,69,Rhône,Auvergne-Rhône-Alpes
4,Toulouse,43.6045,1.4440,France,FR,Occitanie,504078,504078,Toulouse,Toulouse,31,Haute-Garonne,Occitanie


In [21]:
# Sort df merged by 'city' column
df_merged.sort_values(by='city', inplace=True)

In [23]:
df_merged.head()

,city,lat,lng,country,iso2,admin_name,population,population_proper,cleaned_city_name,Commune,Département (numéro),Département (nom),Région
580,Ablon-sur-Seine,48.7256,2.4211,France,FR,Île-de-France,5946,5946,Ablon-sur-Seine,Ablon-sur-Seine,94,Val-de-Marne,Île-de-France
304,Achères,48.9602,2.0684,France,FR,Île-de-France,21368,21368,Achères,Achères,18,Cher,Centre-Val de Loire
184,Agen,44.2049,0.6212,France,FR,Nouvelle-Aquitaine,32485,32485,Agen,Agen,47,Lot-et-Garonne,Nouvelle-Aquitaine
20,Aix-en-Provence,43.5263,5.4454,France,FR,Provence-Alpes-Côte d’Azur,147478,147478,Aix-en-Provence,Aix-en-Provence,13,Bouches-du-Rhône,Provence-Alpes-Côte d'Azur
187,Aix-les-Bains,45.6886,5.9150,France,FR,Auvergne-Rhône-Alpes,31874,31874,Aix-les-Bains,Aix-les-Bains,73,Savoie,Auvergne-Rhône-Alpes


In [26]:
# sauvegarder le dataframe mergé en CSV:
df_merged.to_csv('../data/merged_cities_communes.csv', index=False)

In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 626 entries, 0 to 626
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   city               626 non-null    object 
 1   lat                626 non-null    float64
 2   lng                626 non-null    float64
 3   country            626 non-null    object 
 4   iso2               626 non-null    object 
 5   admin_name         626 non-null    object 
 6   capital            120 non-null    object 
 7   population         626 non-null    int64  
 8   population_proper  626 non-null    int64  
 9   cleaned_city_name  626 non-null    object 
dtypes: float64(2), int64(2), object(6)
memory usage: 53.8+ KB


In [19]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33479 entries, 0 to 33478
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Commune               33479 non-null  object
 1   Département (numéro)  33479 non-null  object
 2   Département (nom)     33479 non-null  object
 3   Région                33479 non-null  object
 4   cleaned_city_name     33479 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB
